In [1]:
import pickle
import pandas as pd
import numpy as np
import nltk
import torch
import string
import os
import keras
import tensorflow as tf
from string import punctuation
from sklearn import metrics
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TFBertModel
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.resnet50 import ResNet50
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image
from PIL import UnidentifiedImageError
from tensorflow.keras.preprocessing import image

/Users/howardtangkulung/.pyenv/versions/3.10.6/envs/movie_genre_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
big_train_df = pd.read_csv('../raw_data/large_dataset/big_data_train.csv').drop(columns = "Unnamed: 0").head(6000)
big_test_df = pd.read_csv('../raw_data/large_dataset/big_data_test.csv').drop(columns = "Unnamed: 0").head(1000)
big_val_df = pd.read_csv('../raw_data/large_dataset/big_data_val.csv').drop(columns = "Unnamed: 0").head(1000)

In [3]:
big_train_df["imdb_id"][0]

'tt6791350'

In [4]:
# resize all images to 256x256
# convert images to arrays (ignore corrupted images)
def get_image_array(df):
    width, height = 256, 256
    image_array = []
    unidentified_count = 0
    not_found_count = 0
    print(f"total: {df.shape[0]}")
    for i in tqdm(range(df.shape[0])):
        try:
            folder = df["location"][i]
            image_path = f"../raw_data/large_dataset/{folder}/{df['imdb_id'][i]}.jpg"
            img = image.load_img(image_path, target_size=(width, height, 3))
            input_arr = np.asarray(image.img_to_array(img))
            image_array.append([df['imdb_id'][i], input_arr])
        except UnidentifiedImageError as e1:
            unidentified_count += 1
            pass
        except FileNotFoundError as e2:
            not_found_count += 1
            pass
    print(f"{unidentified_count} files were unidentified\n{not_found_count} files were not found")
    print(f"we got {len(image_array)}")
    img_array_df = pd.DataFrame(image_array, columns=["imdb_id", "image_array"])
    df = df.merge(img_array_df, on="imdb_id", how="right").drop(columns=["location"])
    df = df.dropna().reset_index(drop=True)
    return df

In [5]:
df_train = get_image_array(big_train_df)
df_train.head(1)

total: 6000


  0%|          | 0/6000 [00:00<?, ?it/s]

100%|██████████| 6000/6000 [00:19<00:00, 300.18it/s]


0 files were unidentified
0 files were not found
we got 6000


,imdb_id,genre,plot,image_array
0,tt6791350,"['Action', 'Adventure', 'Comedy']",still reeling from the loss of gamora peter qu...,"[[[118.0, 9.0, 186.0], [118.0, 15.0, 192.0], [..."


In [6]:
df_test = get_image_array(big_test_df)
df_val = get_image_array(big_val_df)

total: 1000


100%|██████████| 1000/1000 [00:03<00:00, 299.40it/s]


0 files were unidentified
0 files were not found
we got 1000
total: 1000


100%|██████████| 1000/1000 [00:03<00:00, 302.23it/s]


0 files were unidentified
0 files were not found
we got 1000


# Preprocess Data

In [7]:
def preprocess_genre(df):
    df["genre"] = (
        df["genre"]
        .apply(eval)
        .apply(lambda x: [genre.strip() for genre in x])
        )
    multilabel_binarizer = MultiLabelBinarizer()
    multilabel_binarizer.fit(df['genre'])

    #transform target variable
    y = multilabel_binarizer.transform(df['genre'])
    genre_names = multilabel_binarizer.classes_

    # Adding
    for i in range(len(genre_names)):
        df[f"{genre_names[i]}"] = y[:,i]
    
    return (df, y)
    

In [8]:
df_train, y_train = preprocess_genre(df_train)
df_train.head(1)

,imdb_id,genre,plot,image_array,Action,Adventure,Animation,Biography,Comedy,Crime,Horror,Romance,Thriller,War
0,tt6791350,"[Action, Adventure, Comedy]",still reeling from the loss of gamora peter qu...,"[[[118.0, 9.0, 186.0], [118.0, 15.0, 192.0], [...",1,1,0,0,1,0,0,0,0,0


In [7]:
df_train, y_train = preprocess_genre(df_train)
df_train.head(1)
y_train

,imdb_id,genre,plot,image_array
0,tt6791350,"[Action, Adventure, Comedy]",still reeling from the loss of gamora peter qu...,"[[[118.0, 9.0, 186.0], [118.0, 15.0, 192.0], [..."


In [9]:
y_train.shape

(6000, 10)

In [10]:
df_test["genre"] = (
    df_test["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )
df_val["genre"] = (
    df_val["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [11]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_train['genre'])

#transform target variable
y_train = multilabel_binarizer.transform(df_train['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_train[f"{genre_names[i]}"] = y_train[:,i]

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_test['genre'])

#transform target variable
y_test = multilabel_binarizer.transform(df_test['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_test[f"{genre_names[i]}"] = y_test[:,i]

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_val['genre'])

#transform target variable
y_val = multilabel_binarizer.transform(df_val['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_val[f"{genre_names[i]}"] = y_val[:,i]

In [12]:
print(y_test.shape, y_train.shape, y_val.shape)

(1000, 10) (6000, 10) (1000, 10)


In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_text = df_train['plot'].to_list()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

test_text = df_test['plot'].to_list()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

val_text = df_val['plot'].to_list()

In [14]:
def tokenize_encode_multimodal(df):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    text = df['plot'].to_list()
    encodings = tokenizer(text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
    return encodings

In [19]:
tokenize_encode_multimodal(df_train)

{'input_ids': array([[  101,  2145, 28515, ...,  2162, 20286,   102],
       [  101,  1037,  2470, ..., 26836, 15166,   102],
       [  101,  2661, 17103, ...,  4154,  2332,   102],
       ...,
       [  101,  1037,  6887, ...,  2004,  9298,   102],
       [  101,  2048,  2814, ...,     0,     0,     0],
       [  101,  1037,  2402, ...,  5856,  2008,   102]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])}

In [15]:
train_encodings = tokenizer(train_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
test_encodings = tokenizer(test_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
val_encodings = tokenizer(val_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")

In [16]:
# load the dataset and split into train/test sets
# REMEMBER TO DROP COLUMNS!!
X_train_img = np.array(list(df_train["image_array"].values))
X_train_text = train_encodings['input_ids']
y_train = y_train
X_test_img = np.array(list(df_test["image_array"].values))
X_test_text = test_encodings['input_ids']
y_test = y_test
X_val_img = np.array(list(df_val["image_array"].values))
X_val_text = val_encodings['input_ids']
y_val = y_val

In [21]:
type(y_train)

numpy.ndarray

# RESNET 50

In [14]:
def load_model():
    model = ResNet50(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
    return model


In [15]:
def set_nontrainable_layers(model):
    model.trainable = False
    return model

In [16]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(512, activation='relu')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    return model

# CNN

In [17]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(256,256,3)))

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(21, activation=''))

    return model

# BERT Model

In [17]:
# define the BERT-based text feature extractor
def build_text_model():
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_word_ids')
    outputs = bert_model(inputs)[1]
    text_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return text_model

# Multimodal Model

In [18]:
# define the multimodal document classification model
def build_multimodal_model(num_classes):
    model = load_model()
    model = set_nontrainable_layers(model)
    img_model = add_last_layers(model)
    text_model = build_text_model()
    img_input = tf.keras.layers.Input(shape=(256, 256, 3), name='img_input')
    text_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='text_input')
    img_features = img_model(img_input)
    text_features = text_model(text_input)
    concat_features = tf.keras.layers.concatenate([img_features, text_features])
    x = tf.keras.layers.Dense(512, activation='relu')(concat_features)
    x = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    multimodal_model = tf.keras.Model(inputs=[img_input, text_input], outputs=x)
    return multimodal_model

In [24]:
y_train.shape[1]

10

In [19]:
# build the multimodal model
num_classes = 10
multimodal_model = build_multimodal_model(num_classes)
multimodal_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 text_input (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 sequential (Sequential)        (None, 512)          89380724    ['img_input[0][0]']              
                                                                                                  
 model (Functional)             (None, 768)          109482240   ['text_input[0][0]']       

In [20]:
# compile the model and train on the train set
multimodal_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'binary_accuracy', 'categorical_accuracy'])

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history = multimodal_model.fit([(X_train_img, X_train_text)], tf.convert_to_tensor(y_train), epochs=5, batch_size=64, validation_data=([(X_val_img, X_val_text)], tf.convert_to_tensor(y_val)), callbacks = [es])

Epoch 1/5


2023-08-24 21:18:37.481821: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


61/94 [==================>...........] - ETA: 26:07 - loss: 1.8647 - accuracy: 0.2600 - auc: 0.7157 - binary_accuracy: 0.7909 - categorical_accuracy: 0.2600

KeyboardInterrupt: 

In [ ]:
model_file_path = "../raw_data/models/multimodal_model2.h5"
model = models.load_model(model_file_path, custom_objects={"TFBertModel": transformers.TFBertModel})